# Decoradores de métodos e classes
***

Algumas funções construídas por um programador pode sofrer modificações ao longo do desenvolvimento do aplicativo, o que significa que tudo o que um programador escreve deve estar apto a sofrer modificações.

Em python existe uma forma simples de adicionar funcionalidades a uma dada função a partir de outra função DECORADORA.

Decoradores de métodos funcionam assim como decoradores de classe, dessa forma basta criar uma função decoradora e aplica-la a um método usando a sintaxe do @

**Decorator**: É um design pattern que permite adicionar um comportamento a um objeto em tempo de execução, dinamicamente.

* É uma função que recebe outra função como parâmetro, gerando uma nova função que adiciona algumas funcionalidades a função original e a retorna essa nova função.


* Normalmente usado para trechos de códigos que vive se repetindo dentro de diversas funções, podemos criar um decorador para inserir esse trecho de maneira dinamica nessas funções, isso é valido para o principio Don't Repeat Youself.


* Tem a sintaxe de **@alguma_coisa**


* Basicamente um decorator irá pegar sua função e modifica-la.


* Uma mesma função/metodo pode ter mais de um decorador, e é executado na ordem de cima para baixo dos decoradores inseridos

Alguns decoradores de métodos populares:

* **@staticmethod**: Indica que o dado método é estatico, logo deverá ser chamado usando a classe e não uma instância da classe, assim a instância não deve ser passada como argumento. Quando eu decoro com isso eu indico que não preciso de uma instancia (self) sendo passada como argumento do método em sua implementação.


* **@classmethod**: Ao inves de instância ser passada com argumento para o método, a classe será passada como argumento.


* **@property**: Coloca uma propriedade/atributo em modo somente de leitura

Decoradores de classe vão mudar todo o funcionamento de uma dada classe, por exemplo, você pode criar o decorador **singleton** que faz com que apenas uma instância de classe possa ser criada.

Uma classe decoradora consiste numa classe que irá decorar a outra, logo a classe decoradora deverá receber uma classe como argumento, por exemplo, suponhamos que se queira acompanhar os métodos usados por uma determinada classe. Iremos criar uma classe decoradora Trace que permitirá acompanhar a execução de uma classe decorada por ela.

***
### @staticmethod
***

In [1]:
class A(object):
    
    @staticmethod
    def method1(*args):
        # Não preciso do self
        return args
        
    def method2(*args):
        # Preciso do self
        return args

In [2]:
a = A()

In [3]:
print(a.method1(1, 2, 3, 4))
print(a.method2(1, 2, 3, 4))

(1, 2, 3, 4)
(<__main__.A object at 0x7f30d82c4ba8>, 1, 2, 3, 4)


***
### @classmethod
***

In [4]:
class A(object):
    
    @classmethod
    def method1(cls, *args):
        # Preciso do cls
        return args
        
    @classmethod
    def method2(*args):
        # Preciso do cls
        return args

In [5]:
a = A()

In [6]:
print(a.method1(1, 2, 3, 4))
print(a.method2(1, 2, 3, 4))

(1, 2, 3, 4)
(<class '__main__.A'>, 1, 2, 3, 4)


***
### @overrides
***

In [7]:
# Cria o decorador
def overrides(interface):
    
    def decorator(method):
        # Verifica se o nome do método está contido dentro do dicionario
        # que define a classe interface
        if method.__name__ in dir(interface):
            return method
        else:
            raise Exception(
                "Erro na escrita do método {0} o nome correto está na classe {1}"
                .format(method.__name__, interface.__name__)
            )
    
    return decorator

In [8]:
class SuperClass(object):
    
    def method(self):
        pass

In [9]:
class SubClass(SuperClass):
    
    @overrides(SuperClass)
    def method(self):
        print("Olá Marte!")

In [10]:
class Error(SuperClass):
    
    @overrides(SuperClass)
    def methods(self):
        print("Olá Erro!")

Exception: Erro na escrita do método methods o nome correto está na classe SuperClass

***
### Decorador de classe
***

In [11]:
instancias = {}
def singleton(classe):
    """
    Decorador que faz uma classe ter uma única instância
    """
    
    def class_decorator(*args, **kwargs):
        if classe not in instancias:
            instancias[classe] = classe(*args, **kwargs)
        return instancias[classe]
    
    return class_decorator

In [12]:
@singleton
class Person(object):
    
    def __init__(self, name, time, lucre):
        self.name = name
        self.time = time
        self.lucre = lucre
        
    def pay(self):
        return self.time * self.lucre

In [13]:
person = Person("Victor", 40, 10)
print(person.name, person.pay())
person = Person("Sue", 20, 35)
print(person.name, person.pay())

Victor 400
Victor 400


***
### Classe como decorador
***

In [14]:
class Tracer(object):
    
    def __init__(self, classe):
        self.classe = classe
        
    def __call__(self, *args):
        self.wrapped = self.classe(*args)
        return self
    
    # Como trace não tem o método display ele passa pelo metodo __getattr__
    # geattr() retorna o atributo attribute de self.wrapped
    def __getattr__(self, attribute):
        print("Acompanhando: " + attribute)
        return getattr(self.wrapped, attribute)

In [15]:
@Tracer
class Spam(object):
    
    def display(self):
        print("Spam!" * 5)

In [16]:
# Vamos começar os testes
spam = Spam()
spam.display()

Acompanhando: display
Spam!Spam!Spam!Spam!Spam!


***
### Como funciona o Decorator do python
***

Um decorator em Python é uma função que tem como retorno outra função que pode executar um código antes ou depois da função que ele decora.

Ela recebe como parâmetro outra função (poderia ser um método também, não importa) que o nosso decorator irá modificar/melhorar.

Passos:

* Decoretor recebe como parâmetro a função que será adicionada.


* Empacote o método com o wrapper passando como parâmetro os mesmo parâmetros passados para a função, junto com o self para identificar qual o contexto está sendo aplicado caso tenha usado com um método de classe. Se tiver usado com função não precisa do self (nosso caso.)


* Podemos aplicar N decorators na função calcula, porém ela ficará para sempre na função, diferente do decoretor criado com DesignPatterns que será inserido dinamicamente em tempo de execução

In [17]:
def imprime(frase):
    """
    Função simples que imprime uma frase
    """
    
    print(frase)

In [18]:
imprime("Olá")

Olá


In [19]:
# Criando o decorator

def imprime_com_destaque(imprime):
    """
    Chama o método ou função imprime e insere um cabeçalho nele.
    """
    
    def wrapper(frase):
        """
        Empacote o método imprime e retorna o mesmo método com o
        cabeçalho antes e depois da execução do mesmo
        """
    
        # Código executado antes de executar a função
        print("****")

        # Executa a função decorada.
        imprime(frase)

        # Código executado depois de executar a função
        print("****")
    
    # Retorna o empacotador
    return wrapper


In [20]:
# Função decorada

@imprime_com_destaque
def imprime(frase):
    """
    Função simples que imprime uma frase
    """
    print(frase)

In [21]:
imprime('Olá')

****
Olá
****


***

In [22]:
# Criando um decorator que substitua por completo a função original
def header(function):

    def print_header(name):
        print("**********************")
        print("-- Bem Vindo %s --" % name)
        print("**********************")
        return True
    
    return print_header

In [23]:
# Ao utilizar o decorator header, ao inves de imprimir
# Meu nome é Victor irá imprimir o cabeçalho definido no decorator
@header
def hello_world(name):
    print("Meu nome é %s" % name)

In [24]:
# Com isso definimos que o hello_world é a função header
hello_world("Victor")

**********************
-- Bem Vindo Victor --
**********************


True

***

In [25]:
# Criando um decorator que adicione funcionalidades a função original
def header(function):

    def print_header(name):
        print("*********************")
        print("   -- Bem Vindo --   ")
        print("*********************")
        function(name)
    
    return print_header

In [26]:
# Ao utilizar o decorator header, iremos imprimir o
# cabeçalho definido no decorator e o definido no hello_world
@header
def hello_world(name):
    print("Meu nome é %s" % name)

In [27]:
# Com isso definimos que o hello_world é a função header
hello_world("Victor")

*********************
   -- Bem Vindo --   
*********************
Meu nome é Victor


***

In [28]:
# O que acontece por debaixo dos panos
def header(function):

    def print_header(name):
        print("*********************")
        print("   -- Bem Vindo --   ")
        print("*********************")
        function(name)
    
    return print_header

In [29]:
def hello_world(name):
    print("Meu nome é %s" % name)

In [30]:
hello_world = header(hello_world)
hello_world("Victor")

*********************
   -- Bem Vindo --   
*********************
Meu nome é Victor


***

In [31]:
# Aninhamento de decoradores
def p_decorator(function):
    
    def decorator(number1, number2):
        return "<p>" + function(number1, number2) + "</p>"
    
    return decorator

In [32]:
def div_decorator(function):
    
    def decorator(number1, number2):
        return "<div>" + function(number1, number2) + "</div>"
    
    return decorator

In [33]:
def strong_decorator(function):
    
    def decorator(number1, number2):
        return "<strong>" + function(number1, number2) + "</strong>"
    
    return decorator

In [34]:
@p_decorator
@div_decorator
@strong_decorator
def resultHTML(number1, number2):
    return "Resultado: %s" % (number1 * number2)

In [35]:
print(resultHTML(2, 3))

<p><div><strong>Resultado: 6</strong></div></p>


***

In [36]:
# Decoradores com argumentos
def tags(tag):
    # Recebe os argumentos do decorador
    def tags_decorator(function):
        # Decorador da função
        def decorator(number1, number2):
            # Função Decoradora
            return "<{0}>{1}</{2}>".format(tag, function(number1, number2), tag)   
        return decorator
    return tags_decorator

In [37]:
@tags("p")
@tags("div")
@tags("strong")
def resultHTML(number1, number2):
    return "Resultado: %s" % (number1 * number2)

In [38]:
print(resultHTML(2, 3))

<p><div><strong>Resultado: 6</strong></div></p>
